In [86]:
import pandas as pd

In [145]:
# load all the data

In [ ]:
plaintiff_claims=pd.read_excel('plaintiff_claims.xlsx')
plaintiff_descriptions=pd.read_excel('plaintiff_descriptions.xlsx')

In [129]:
# data is the dictionary for all the patents, the key is the patent number, and value is a dictionary of each part of a patent 
data = {}

In [130]:
# this function will update the dictionary data, filling it with new patent content from the input dataframe
# df is the claims data and df_desc is the descriptions data
def preprocess_adding(df,df_desc,data_dict):
    # traverse each patent number
    for idx, patent_number in enumerate(df_desc['Patent_Number']):
        if patent_number not in data.keys():
            data[patent_number] = dict()
        
        # Fill the abstract
        data[patent_number]['Abstract']=df_desc['Abstract'][idx]
        
        # Fill the Claim Part
        for jdx, claim in enumerate(df[df['Patent_Number']==patent_number]['Claim_Set']):
            name = 'Claim_Set_' + str(jdx+1)
            data[patent_number][name] = claim
    
        # Fill the Figure Description
        filepath_figure=df_desc.iloc[idx, df_desc.columns.get_loc('Figures')]
        lines=''
        try:
            with open(filepath_figure, 'rb') as f:
                lines = f.read()
        except FileNotFoundError:
            pass

        if lines:
            data[patent_number]['Figures_Desc']=lines
            
        # Fill the Disclosures
        filepath_figure=df_desc.iloc[idx, df_desc.columns.get_loc('Disclosure')]
        lines=''
        try:
            with open(filepath_figure, 'rb') as f:
                lines = f.read()
        except FileNotFoundError:
            pass
        
        if lines:
            data[patent_number]['Disclosure']=lines

In [ ]:
preprocess_adding(plaintiff_claims,plaintiff_descriptions,data)

In [39]:
import json

In [143]:
# this function save dict to JSON file
def save_to_json(data,filename):
    
    class MyEncoder(json.JSONEncoder):
        def default(self, obj):
            if isinstance(obj, np.ndarray):
                return obj.tolist()
            elif isinstance(obj, bytes):
                return str(obj, encoding='utf-8');
            return json.JSONEncoder.default(self, obj)
    
    data_json = json.dumps(data,cls=MyEncoder,indent=4)
    with open(filename, 'w') as json_file:
        json_file.write(data_json)

In [144]:
# save to json file
filename = 'patent_data.json'
save_to_json(data,filename)